In [1]:
import geopandas as gpd

import sys
sys.path.append('../Mask_RCNN')
import mrcnn.model as modellib

# Autoria
import canopy_detection_4b as orca

In [2]:
logs = '../Mask_RCNN/logs'
config = orca.OrangeCanopyConfig()
dataset = '../datasets/area_estudo_teste1/'

In [3]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Instructions for updating:
Use fn_output_signature instead


In [4]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20211014T2316/mask_rcnn_oranges_trees_canopy_0060.h5'

model.load_weights(weights, by_name=True)

Re-starting from epoch 60


In [5]:
img_test = gpd.read_file(dataset + '/img_grid.geojson')
ids = list(img_test.loc[img_test['split_samples'] == 'test'].index)
len(ids)

939

In [6]:
results = orca.prediction(model, dataset, id_list=ids, verbose=0, first=False, join=True)

Inferincdo copas:   0%|                                                                        | 0/938 [00:00<?, ?it/s]C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
Inferincdo copas: 100%|██████████████████████████████████████████████████████████████| 938/938 [04:38<00:00,  3.37it/s]


Gerando arquivo de saida


AttributeError: 'NoneType' object has no attribute 'type'

In [ ]:
results

In [27]:
import os
import pandas as pd 
from tqdm import tqdm
from time import sleep

centroid_dist = 1
limiar_ovelap = 0.6

shape_path_aux = '../datasets/area_estudo_teste1/results/'

result_paths = []
for filename in os.listdir(shape_path_aux):
    if os.path.splitext(filename)[1].lower() == '.geojson':
        result_paths.append(os.path.join(shape_path_aux, filename))

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols = pols[pols.geometry.is_valid]
pols['geometry'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)
pols = pols.explode('geometry').reset_index(drop=True)
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= centroid_dist]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

# pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(['union', 'centroid'], axis=1, inplace=True)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid

# Gerar intersecoes entre os poligonos encontrados
# Uso de busca indexada (R-tree) com a função gpd.overlay()
over = gpd.overlay(pols, pols, how='intersection')
# Remover poligonos vazios (talvez possa ser dispensado a partir de agora)
over = over.loc[-over['geometry'].is_empty]
# Remover intersecoes originadas pela sobreposição da feição com ela mesma
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)

delete_lines = []
# Barra de progresso #
with tqdm(total=len(over['id_1'])) as pbar:
    pbar.set_description("Simplificando - Micro")
    sleep(0.1)
    # Para cada poligono..
    for i in over['id_1']:
        # intersecções entre o poligono avaliado e todas as outras feições
        intersec = over[over['id_1'] == i]

        # Caso alguma área de intersecao avaliada seja maior que 60% (limiar_overlap) do proprio poligono avaliado
        # ou se poligono avaliado é 3,5x menor que aquele com o qual existe intersecao, o poligono será excluido
        if intersec.loc[intersec['geometry'].area >= limiar_ovelap * pols.loc[i, 'geometry'].area].any(axis=None) or \
                (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
            delete_lines.append(i)

        pbar.update(1)

    # Identificar poligonos a serem apagados
    delete_lines = (list(set(delete_lines)))
# Fim da barra de progresso #

pols.drop(labels=delete_lines, axis=0, inplace=True)
pols = pols.explode().reset_index(drop=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index

Simplificando - Micro: 100%|██████████████████████████████████████████████████████| 4228/4228 [00:21<00:00, 194.87it/s]


In [29]:
pols.to_file('../datasets/area_estudo_teste1/canopy_detection_result.geojson', driver='GeoJSON')

In [8]:
all_files = [gpd.read_file(i) for i in result_paths]

In [9]:
len(all_files)

897

In [10]:
pols = pd.concat(all_files, axis=0).reset_index(drop=True)

In [21]:
pols

,score,detection_score,geometry
0,None,0.999820,"POLYGON ((288715.960 7515119.829, 288716.025 7..."
1,None,0.998830,"POLYGON ((288718.307 7515128.201, 288718.304 7..."
2,None,0.983995,"POLYGON ((288715.546 7515122.025, 288715.545 7..."
3,None,0.963599,"POLYGON ((288710.536 7515117.851, 288710.543 7..."
4,None,0.952288,"POLYGON ((288719.063 7515116.731, 288719.067 7..."
...,...,...,...
16235,None,0.999790,"POLYGON ((289440.273 7515728.740, 289440.413 7..."
16236,None,0.999785,"POLYGON ((289445.319 7515727.748, 289445.384 7..."
16237,None,0.999378,"POLYGON ((289439.913 7515719.433, 289439.845 7..."
16238,None,0.999115,"POLYGON ((289441.410 7515721.639, 289441.482 7..."


In [22]:
pols = pols[pols.geometry.is_valid]
pols

,score,detection_score,geometry
0,None,0.999820,"POLYGON ((288715.960 7515119.829, 288716.025 7..."
1,None,0.998830,"POLYGON ((288718.307 7515128.201, 288718.304 7..."
2,None,0.983995,"POLYGON ((288715.546 7515122.025, 288715.545 7..."
3,None,0.963599,"POLYGON ((288710.536 7515117.851, 288710.543 7..."
4,None,0.952288,"POLYGON ((288719.063 7515116.731, 288719.067 7..."
...,...,...,...
16235,None,0.999790,"POLYGON ((289440.273 7515728.740, 289440.413 7..."
16236,None,0.999785,"POLYGON ((289445.319 7515727.748, 289445.384 7..."
16237,None,0.999378,"POLYGON ((289439.913 7515719.433, 289439.845 7..."
16238,None,0.999115,"POLYGON ((289441.410 7515721.639, 289441.482 7..."


In [19]:
pols.geometry.is_valid[-pols.geometry.is_valid]

12208    False
13635    False
dtype: bool

In [ ]:
pols['geometry_n'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)

In [ ]:
pols['geometry_n'].is_empty.any()

In [ ]:
pols.geometry_n.is_empty[pols.geometry_n.is_empty]

In [ ]:
for i in range(len(all_files)):
    if all_files[i].is_empty.all():
        print(i)

In [ ]:
filtered_files = [gpd.read_file(i) for i in result_paths if gpd.read_file(i)]